# View sample predictions

Trained model from checkpoint

In [7]:
import argparse
import random
import sys
import warnings
import yaml

from transformers.models.mt5 import MT5Tokenizer
import pandas as pd

sys.path.append('..')  # Allow import of project packages
from text_summarizer.data.motions_data_module import MotionsDataModule
from text_summarizer.models import t5
from text_summarizer.lit_models import MT5LitModel
from text_summarizer.util import summarize

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 500)

BASE_PATH = "../training/logs/lightning_logs/version_1/"
MODEL_PATH = BASE_PATH + "checkpoints/epoch=009-val_loss=1.242.ckpt"
HPARAMS_PATH =  BASE_PATH + "hparams.yaml"
RANDOM_STATE = 1

In [5]:
# Load pytorch-lightning experiment args
with open(HPARAMS_PATH, "r") as hparams_file:
   lightning_config = argparse.Namespace(**yaml.load(hparams_file, Loader=yaml.Loader))

dataset = MotionsDataModule(lightning_config)
dataset.prepare_data()
dataset.setup()
print(dataset)

Filtered 618 rows with missing values.
Filtered 7444 texts shorter than 150 characters.
Number of rows remaining: 161989
Using 161989 of 161989 examples.
Train: 113392, Val: 24298, Test: 24299


### Load model

In [6]:
# Instanciate model to pass to lit_model
model = t5.MT5(data_config={}, args=lightning_config)

lit_model = MT5LitModel.load_from_checkpoint(
    checkpoint_path=MODEL_PATH,
    model=model,
)
lit_model.eval()

tokenizer = MT5Tokenizer.from_pretrained(model.model_name)

### Load data

In [59]:
def summarize(model, text, tokenizer, text_max_num_tokens, summary_max_num_tokens):
    text_encoding = tokenizer(
        text,
        max_length=text_max_num_tokens,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )

    generated_ids = model.model.generate(
        input_ids=text_encoding["input_ids"],
        attention_mask=text_encoding["attention_mask"],
        max_length=summary_max_num_tokens,
        num_beams=2,
        repetition_penalty=5.0,
        length_penalty=2.0,
        early_stopping=True,
    )

    preds = [
        tokenizer.decode(
            gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        for gen_id in generated_ids
    ]

    return "".join(preds)

In [60]:
def show_sample_pred(tokenizer):
    sample_index = random.randint(0, len(dataset.data_test.data))
    text = dataset.data_test.data[sample_index]
    true_summary = dataset.data_test.targets[sample_index]
    model_summary = summarize(
        model=model,
        text=text,
        tokenizer=tokenizer,
        text_max_num_tokens=512,
        summary_max_num_tokens=64
    )
    print("Motion text:")
    print(50*"-")
    print(text[:500])
    print(50*"-")
    print("Actual title:")
    print(true_summary)
    print(50*"-")
    print("Predicted title:")
    print(model_summary)
    print(50*"-")

show_sample_pred(lit_model.tokenizer)

Motion text:
--------------------------------------------------
Vi har i vår motion om den allmänna inriktningen av den ekonomiska politiken m. m. bl. a. föreslagit att-som ett led i ett stöd till de särskilt utsatta grupperna - en temporär höjning skall ske av studiebidraget inom studiehjälpssystemet och det förlängda barnbidraget. Den temporära höjningen bör utgå med nio tolftedelar av det föreslagna beloppet för extra tillägg till barnbidraget (500 kr.), dvs. 375 kr. Utbetalningen bör göras så snart som möjligt efter början av höstterminen 1978 och ha for
--------------------------------------------------
Actual title:
med anledning av propositionen 1977/78:150 med förslag till slutlig reglering av statsbudgeten för budgetåret 1978/79, m. m. (kompletteringsproposition)
--------------------------------------------------
Predicted title:
om vidgad rätt till studiebidrag
--------------------------------------------------
